# Title: ...

## 0. Imports

In [1]:
# Notebook similar to: https://www.kaggle.com/code/gracehephzibahm/prediction-of-rent-prices-in-barcelona
# Web scraper with good insights and EDA: https://github.com/agonzalezramos/Idealista-Price-Prediction/tree/main

# Dataset pisos.csv https://www.kaggle.com/datasets/thedevastator/analysis-of-spanish-apartment-pricing-and-size-p (cita: https://zenodo.org/records/4263693)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Load Data

In [3]:
# Load Data
og_df = pd.read_csv('../data/pisos_clean.csv', sep=',')

C:\Users\pdpau\AppData\Local\Temp\ipykernel_3528\4198264111.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  og_df = pd.read_csv('../data/pisos_clean.csv', sep=',')


## 2. Data Preparation

In [4]:
df = og_df.copy()

df.head(5)

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
0,Casa en calle Urb. Las Mimosas,Ames (San Tome),250.000 €,315 m²,4,793 €/m²,3,31,pisos,a_coruna
1,Piso en Milladoiro,Ames (San Tome),90.000 €,72 m²,2,1.250 €/m²,1,11,pisos,a_coruna
2,Casa en calle La Piedra,A Ortigueira (Cariño),50.000 €,92 m²,2,543 €/m²,1,40,pisos,a_coruna
3,Casa en Perillo,Perillo (Oleiros),359.000 €,244 m²,4,1.471 €/m²,3,39,pisos,a_coruna
4,Chalet en Urbanización Las Mimosas,O Milladoiro (Ames),200.430 €,315 m²,4,636 €/m²,3,24,pisos,a_coruna


In [5]:
# Drop NaN values
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 719459 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     719459 non-null  object
 1   location    719459 non-null  object
 2   price       719459 non-null  object
 3   size        719459 non-null  object
 4   rooms       719459 non-null  object
 5   price/m2    719459 non-null  object
 6   bathrooms   719459 non-null  object
 7   Num Photos  719459 non-null  object
 8   type        719459 non-null  object
 9   region      719459 non-null  object
dtypes: object(10)
memory usage: 60.4+ MB


In [6]:
# Recuperar csv de regions catalanes filtrat manualment
catalan_regions = pd.read_csv('../data/catalan_regions.csv')
catalan_regions_list = list(catalan_regions['0'])
catalan_regions_list

# Filtrar per regions catalanes
df_cat = df[df['region'].isin(catalan_regions_list)]
df_cat

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
20983,"Piso en Carrer del Bruc, cerca de Carretera de...",Valldaura-Plaça Catalunya (Manresa),85.000 €,87 m²,4,977 €/m²,1,10,pisos,barcelona
20984,"Piso en Carrer de les Orenetes, 37",Viladecavalls,149.900 €,57 m²,2,2.629 €/m²,1,16,pisos,barcelona
20985,"Casa en Carrer de la Creu, cerca de Carrer de ...",Sant Just Desvern,1.095.000 €,371 m²,6,2.951 €/m²,4,46,pisos,barcelona
20986,Piso en Centre-Zona Estació-Passeig de Joan Ma...,Centre-Zona Estació-Passeig de Joan Maragall (...,297.000 €,112 m²,3,2.651 €/m²,2,30,pisos,barcelona
20987,"Piso en Carrer del Retir, 4",Castellar del Vallès,230.000 €,70 m²,3,3.285 €/m²,1,1,pisos,barcelona
...,...,...,...,...,...,...,...,...,...,...
777170,"Finca rústica en Carrer Bosc, nº 51",Lliçà d'Amunt,1.100.000 €,1.120 m²,10,982 €/m²,1.0,32,fincas_rusticas,valles_oriental_sud
777171,Finca rústica en Els Dominics,Cardedeu,1.500.000 €,520 m²,8,2.884 €/m²,3.0,12,fincas_rusticas,valles_oriental_sud
777176,Finca rústica en La Roca del Vallès,La Roca del Vallès,1.350.000 €,350 m²,5,3.857 €/m²,2.0,9,fincas_rusticas,valles_oriental_sud
777177,"Finca rústica en Bajada Les Valls, nº S/N",La Roca del Vallès,456.000 €,202 m²,5,2.257 €/m²,3.0,30,fincas_rusticas,valles_oriental_sud


## 3. Exploratory Data Analysis

In [7]:
df_cat.head(5)

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
20983,"Piso en Carrer del Bruc, cerca de Carretera de...",Valldaura-Plaça Catalunya (Manresa),85.000 €,87 m²,4,977 €/m²,1,10,pisos,barcelona
20984,"Piso en Carrer de les Orenetes, 37",Viladecavalls,149.900 €,57 m²,2,2.629 €/m²,1,16,pisos,barcelona
20985,"Casa en Carrer de la Creu, cerca de Carrer de ...",Sant Just Desvern,1.095.000 €,371 m²,6,2.951 €/m²,4,46,pisos,barcelona
20986,Piso en Centre-Zona Estació-Passeig de Joan Ma...,Centre-Zona Estació-Passeig de Joan Maragall (...,297.000 €,112 m²,3,2.651 €/m²,2,30,pisos,barcelona
20987,"Piso en Carrer del Retir, 4",Castellar del Vallès,230.000 €,70 m²,3,3.285 €/m²,1,1,pisos,barcelona


## 4. Model

In [8]:
# Data Cleaning
df_cat['price'] = df_cat['price'].str.replace(' €', '')
df_cat['price'] = df_cat['price'].str.replace('.', '')
# Convertir a numérico manejando errores
df_cat['price'] = pd.to_numeric(df_cat['price'], errors='coerce')
# Opcionalmente, eliminar filas con valores NaN si se generaron
df_cat = df_cat.dropna(subset=['price'])
# Convertir a entero
df_cat['price'] = df_cat['price'].astype(int)

df_cat['size'] = df_cat['size'].str.replace(' m²', '')
df_cat['size'] = pd.to_numeric(df_cat['size'], errors='coerce')
df_cat = df_cat.dropna(subset=['size'])
df_cat['size'] = df_cat['size'].astype(int)

df_cat['price/m2'] = df_cat['price'] / df_cat['size']
df_cat['price/m2'] = df_cat['price/m2'].astype(float)

df_cat.drop(columns=['summary', 'Num Photos'], inplace=True)

df_cat['location'] = df_cat['location'].astype(str)
df_cat['rooms'] = df_cat['rooms'].astype(int)
df_cat['bathrooms'] = df_cat['bathrooms'].astype(int)
df_cat['type'] = df_cat['type'].astype(str)
df_cat['region'] = df_cat['region'].astype(str)

C:\Users\pdpau\AppData\Local\Temp\ipykernel_3528\2024785018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['price'] = df_cat['price'].str.replace(' €', '')
C:\Users\pdpau\AppData\Local\Temp\ipykernel_3528\2024785018.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['price'] = df_cat['price'].str.replace('.', '')
C:\Users\pdpau\AppData\Local\Temp\ipykernel_3528\2024785018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [9]:
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180504 entries, 20983 to 777178
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   location   180504 non-null  object 
 1   price      180504 non-null  int32  
 2   size       180504 non-null  int32  
 3   rooms      180504 non-null  int32  
 4   price/m2   180504 non-null  float64
 5   bathrooms  180504 non-null  int32  
 6   type       180504 non-null  object 
 7   region     180504 non-null  object 
dtypes: float64(1), int32(4), object(3)
memory usage: 9.6+ MB


In [10]:
# Label encoding
from sklearn.preprocessing import LabelEncoder

#df.drop(columns=['location'], inplace=True)
le_location = LabelEncoder()
le_type = LabelEncoder()

df_cat.drop(columns=['region'], inplace=True)
#le_region = LabelEncoder()

df_cat['location'] = le_location.fit_transform(df_cat['location'])
df_cat['type'] = le_type.fit_transform(df_cat['type'])
#df['region'] = le_region.fit_transform(df['region'])

In [ ]:
#df_cat['location'].value_counts()

# Drop locations with less than 10 ocurrences
df_cat = df_cat.groupby('location').filter(lambda x: len(x) > 10)

#df_cat['location'].value_counts()

#df_cat

In [12]:
# Model
from sklearn.model_selection import train_test_split

X = df_cat.drop(columns=['price'])
y = df_cat['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

len(X_train), len(X_test), len(y_train), len(y_test)

(133578, 44526, 133578, 44526)

In [13]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## 5. Evaluation

In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('R2 Score:', r2_score(y_test, y_pred))

Mean Squared Error: 65888403.97797642
Mean Absolute Error: 744.999572160086
R2 Score: 0.9994962533553584


## 6. Explainability